In [ ]:
%pip install openai

In [1]:
import pandas as pd
import json
import numpy as np
from time import sleep
import time
from openai import OpenAI
from tqdm import tqdm
client = OpenAI(api_key=open('API_key').read())

In [2]:
# Load the test datasets from the GitHub repositories (access to them is obtained by request to the AGILE repository owner)

en_ginco = pd.read_json("../../datasets/EN-GINCO-test-dataset/EN-GINCO.jsonl", lines=True)
x_ginco = pd.read_json("../../datasets/X-GINCO-test-set/X-GINCO.jsonl", lines=True)

print(en_ginco.shape, x_ginco.shape)

(272, 4) (790, 6)


In [3]:
en_ginco.head(2)

,text,labels,dataset,language
0,Welcome to KBismarck.org! This is a community ...,Information/Explanation,EN-GINCO,English
1,Why graft thrives in postconflict zones <p> A ...,News,EN-GINCO,English


In [6]:
def predict_gpt(df_test_name, gpt_model):

	dfs = {
		"en-ginco": en_ginco,
		"x-ginco": x_ginco
	}

	df = dfs[df_test_name]

	responses = []
	
	texts = df["text"].to_list()

	start_time = time.time()

	labels_dict = {
		0: "Other",
		1: "Information/Explanation",
		2: "News",
		3: "Instruction",
		4: "Opinion/Argumentation",
		5: "Forum",
		6: "Prose/Lyrical",
		7: "Legal",
		8: "Promotion"
	}

	for text in texts:
		# the "v5" models do not have the "temperature" parameter
		if "gpt-5" not in gpt_model:
			completion = client.chat.completions.create(model=gpt_model,
			response_format= {"type": "json_object"},
			messages= [
			{
				"role": "user",
				"content": f"""
				### Task
				Your task is to classify the following text according to genre. Genres are text types, defined by the function of the text, author’s purpose and form of the text. Always provide a label, even if you are not sure.

				### Output format
					Return a valid JSON dictionary with the following key: 'genre' and a value should be an integer which represents one of the labels according to the following dictionary: {labels_dict}.

					
					Text: '{text}'
			"""
				}
			],
			temperature = 0)
		# the "v5" models do not have the "temperature" parameter
		elif "gpt-5" in gpt_model:
			completion = client.chat.completions.create(model=gpt_model,
			response_format= {"type": "json_object"},
			messages= [
			{
				"role": "user",
				"content": f"""
				### Task
				Your task is to classify the following text according to genre. Genres are text types, defined by the function of the text, author’s purpose and form of the text. Always provide a label, even if you are not sure.

				### Output format
					Return a valid JSON dictionary with the following key: 'genre' and a value should be an integer which represents one of the labels according to the following dictionary: {labels_dict}.

					
					Text: '{text}'
			"""
				}
			],
		)
		else:
			print("The model is not supported, check the code.")

		response=completion.choices[0].message.content

		response = response.replace("\n", "")
		response = response.replace("\t", "")

		# Convert the string into a dictionary
		response = json.loads(response)

		# Get out a label
		try:
			predicted = labels_dict[response["genre"]]
			responses.append(predicted)
		# add a possibility of something going wrong
		except:
			predicted = "error"
			print("error with extracting a label")
			responses.append(predicted)

	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for {df.shape[0]} instances - {elapsed_time_min/df.shape[0]} s per instance.")

	# Create a json with results

	current_results = {
		"system": gpt_model,
		"predictions": [
			{
			"train": "X-GENRE (train split)",
			"test": "{}".format(df_test_name),
			"predictions": responses,
			}
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(gpt_model, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(gpt_model, df_test_name))

In [ ]:
for test in ["en-ginco", "x-ginco"]:
	for model in ["gpt-4o-2024-08-06", "gpt-3.5-turbo-0125", "gpt-4o-mini-2024-07-18"]:
		print(model)
		predict_gpt(test, model)

In [7]:
# Evaluate the new v5 models
for test in ["en-ginco", "x-ginco"]:
	for model in ["gpt-5-nano-2025-08-07", "gpt-5-mini-2025-08-07","gpt-5"]:
		print(model)
		predict_gpt(test, model)

gpt-5-nano-2025-08-07
Prediction finished. It took 11.63731210231781 min for 272 instances - 2.567054140217164 s per instance.
Classification with gpt-5-nano-2025-08-07 on en-ginco finished.
gpt-5-mini-2025-08-07
Prediction finished. It took 10.601998893419902 min for 272 instances - 2.3386762264896843 s per instance.
Classification with gpt-5-mini-2025-08-07 on en-ginco finished.
gpt-5
Prediction finished. It took 20.794515120983124 min for 272 instances - 4.587025394334512 s per instance.
Classification with gpt-5 on en-ginco finished.
gpt-5-nano-2025-08-07
Prediction finished. It took 35.32288513183594 min for 790 instances - 2.682750769506527 s per instance.
Classification with gpt-5-nano-2025-08-07 on x-ginco finished.
gpt-5-mini-2025-08-07
Prediction finished. It took 36.89934118191401 min for 790 instances - 2.802481608752963 s per instance.
Classification with gpt-5-mini-2025-08-07 on x-ginco finished.
gpt-5
Prediction finished. It took 53.55667865673701 min for 790 instances -